In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.db_agg
people = db.people
print('connexion success')


connexion success


In [4]:
db.people.insert([
	{"name" : "Ada", "age" : 20},
	{"name" : "Fred", "age" : 35},
	{"name" : "Susan", "age" : 60},
	{"name" : "Andy", "age" : 35}
])

<ipython-input-4-2559ea3c6ce4>:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  db.people.insert([


[ObjectId('600fd10f6b61282c2b8e5715'),
 ObjectId('600fd10f6b61282c2b8e5716'),
 ObjectId('600fd10f6b61282c2b8e5717'),
 ObjectId('600fd10f6b61282c2b8e5718')]

In [2]:
# Qry classique SQL
# SELECT name FROM people WHERE age = 35 ORDER BY name

request = people.find({"age": 35}, {"name": 1, "_id": 0}).sort("name")
for pp in request:
    print(pp)

{'name': 'Andy'}
{'name': 'Fred'}


In [3]:
# La Qry avec aggregate => même résultat
# La fonction "aggregate()" permet de spécifier des chaînes d’opérations, appelées pipeline d’agrégation ou encore stage.

request = db.people.aggregate( [
    { "$match" : {
        "age":35
    }},
    { "$project" : {
        "name":1, "_id":0
    }},
    { "$sort" : {
        "name":1
    }}
] )
for pp in request:
    print(pp)

{'name': 'Andy'}
{'name': 'Fred'}


In [4]:
# Q : Cmnt alléger l'écrirure de la Qry ?
# R1 : Passer par des vars !!!

varMatch = { "$match" : { "age":35} }
varProject = { "$project" : {"name":1, "age":1, "_id":0} }
varSort = { "$sort" : {"name":1} }

request = db.people.aggregate( [varMatch, varProject, varSort] )
for pp in request:
    print(pp)

{'name': 'Andy', 'age': 35}
{'name': 'Fred', 'age': 35}


In [26]:
# Exo Restaurants
# Q : La répartition des restaurants par quartier ?

restaurants = db.restaurants

request = restaurants.aggregate( [
    { "$group" : {
        "_id": "$borough",
        "nbResto": {"$sum": 1}
    }}
] )
for pp in request:
    print(pp)

{'_id': 'Bronx', 'nbResto': 2338}
{'_id': 'Manhattan', 'nbResto': 10258}
{'_id': 'Missing', 'nbResto': 51}
{'_id': 'Staten Island', 'nbResto': 969}
{'_id': 'Queens', 'nbResto': 5656}
{'_id': 'Brooklyn', 'nbResto': 6085}


In [27]:
# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?

match_grade_C = {"$match": {"grades.0.grade": "C"}}
project = {"$project": {"name": 1, "borough": 1, "_id": 0}}
tri = {"$sort": {"_id": 1}}
repart_quartier = { "$group" : {
        "_id": "$borough",
        "nbResto": {"$sum": 1}
    }}
    
request = restaurants.aggregate([match_grade_C, project, repart_quartier, tri])
for pp in request:
    print(pp)

{'_id': 'Bronx', 'nbResto': 27}
{'_id': 'Brooklyn', 'nbResto': 56}
{'_id': 'Manhattan', 'nbResto': 83}
{'_id': 'Queens', 'nbResto': 47}
{'_id': 'Staten Island', 'nbResto': 7}


In [29]:
# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?

eclater_liste = {"$unwind" : "$grades"}
avg_quartier = {"$group" : {"_id":"$borough","moyenne":{"$avg":"$grades.score"}}}
tri = {"$sort": {"moyenne":-1}}

request = restaurants.aggregate([eclater_liste, avg_quartier, tri])
for pp in request:
    print(pp)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


In [39]:
# Exercice  Aggréger des transactions
# Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes :

# Q-1. Calculer le montant total des paiements ?

transactions = db.transactions

match = {"$match": {"Payment.Success": True}}
group = {"$group": {"_id": "Total", "Total": {"$sum": "$Payment.Total"}}}


request = transactions.aggregate([match, group])
for pp in request:
    print(pp)

{'_id': 'Total', 'Total': 1230}


In [65]:
# Q-2. Calculer le montant total par transactions (montant total du panier) ?

unwind = {"$unwind": "$Panier"}
group = {"$group": {"_id": "$TransactionId", "Total": {"$sum": "$Panier.price"}}}

request = transactions.aggregate([unwind, group])
for pp in request:
    print(pp)

{'_id': 'tran2', 'Total': 410}
{'_id': 'tran1', 'Total': 410}
{'_id': 'tran3', 'Total': 410}
{'_id': 'tran4', 'Total': 410}


In [45]:
# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

match = {"$match": {"Payment.Success": True}}
group = {"$group": {"_id": "$Payment.Type", "Total": {"$sum": "$Payment.Total"}}}


request = transactions.aggregate([match, group])
for pp in request:
    print(pp)

{'_id': 'Debit-Card', 'Total': 410}
{'_id': 'Credit-Card', 'Total': 820}


In [63]:
# Q-4. Trouver l'Id le plus élevé.

sort = {"$sort": {"Id": -1}}
limit = {"$limit": 1}

request = transactions.aggregate([sort, limit])
for pp in request:
    print(pp)

{'_id': ObjectId('60100058ad0f7231a8821eb2'), 'Id': 103, 'Name': 'Dylan', 'TransactionId': 'tran4', 'Panier': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}], 'Subscriber': True, 'Payment': None, 'Note': 'Payment is Null'}


In [71]:
# Q-5. Find the max price (Transaction.price) ?

unwind = {"$unwind": "$Panier"}
project = {"$project": {"Panier.price": 1, "_id": 0}}
sort = {"$sort": {"Panier.price": -1}}
limit = {"$limit": 1}

request = transactions.aggregate([unwind, sort, project, limit])
for pp in request:
    print(pp)

{'Panier': {'price': 210}}


In [73]:
# Distinct
# Q : Age unique => distinct ?

request = people.distinct("age")
for pp in request:
    print(pp)

20
35
60


In [87]:
from bson.code import Code

def mapFunction(): {
   emit(this.name, this.age)
}

def reduceFunction(keyName, valueAge): {
   print (Array.sum(valueAge))
}

request = people.map_reduce(
   mapFunction,
   reduceFunction,
   "map_reduce_example"
)

for pp in request.find():
    print(pp)

InvalidDocument: cannot encode object: <function mapFunction at 0x00000237404253A0>, of type: <class 'function'>